In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pydicom
import os
import matplotlib.pyplot as plt
from glob import glob
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from sklearn.cluster import KMeans
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.figure_factory as ff
from plotly.graph_objs import *
init_notebook_mode(connected=True)
import pandas as pd
from tqdm import tqdm
import seaborn as sns

In [ ]:
df_train = pd.read_csv('/kaggle/input/rsna-str-pulmonary-embolism-detection/train.csv')
df_test = pd.read_csv('/kaggle/input/rsna-str-pulmonary-embolism-detection/test.csv')

In [ ]:
df_train.head(5)

In [ ]:
print('Plotting PE present on images')
plt.hist(df_train['pe_present_on_image'])
plt.xlabel('PE present or not')
plt.ylabel('Number of patients with PE present')

# It is evident from the above image that the Classes for PE present or not is imbalanced

In [ ]:
df_test.head(5)

In [ ]:
print('Number of Studies in the dataset', df_train['StudyInstanceUID'].nunique())


**Let's see how many series in each study**

In [ ]:
seriesIDs = []
seriesIDs_lens = []
studyIDs = df_train['StudyInstanceUID'].unique()
for studyID in tqdm(studyIDs):
    seriesIDs.append(df_train[df_train['StudyInstanceUID'] == studyID]['SeriesInstanceUID'].unique())
    seriesIDs_lens.append(len(df_train[df_train['StudyInstanceUID'] == studyID]['SeriesInstanceUID'].unique()))

In [ ]:
plt.bar(len(seriesIDs_lens),np.array(seriesIDs_lens))
plt.xlabel('Studies')
plt.ylabel('Series Length')

Let's find out how many images in each study

In [ ]:
SOPInstanceIDs = []
SOPInstanceIDs_lens = []
for seriesID in tqdm(seriesIDs):
    SOPInstanceIDs.append(df_train[df_train['SeriesInstanceUID'] == str(seriesID)[2:-2]]['SOPInstanceUID'].unique())
    SOPInstanceIDs_lens.append(len(df_train[df_train['SeriesInstanceUID'] == str(seriesID)[2:-2]]['SOPInstanceUID'].unique()))

In [ ]:
plt.hist(np.array(SOPInstanceIDs_lens))
plt.xlabel('Series IDs')
plt.ylabel('Number of images Length')

In [ ]:
print('Columns in the Training Dataset : \n',list(df_train))

In [ ]:
df_corr = df_train[['pe_present_on_image', 'negative_exam_for_pe', 'qa_motion', 
                    'qa_contrast', 'flow_artifact', 'rv_lv_ratio_gte_1', 'rv_lv_ratio_lt_1', 
                    'leftsided_pe', 'chronic_pe', 'true_filling_defect_not_pe', 'rightsided_pe', 'acute_and_chronic_pe', 
                    'central_pe', 'indeterminate']]
df_corr.head(5)

In [ ]:
corr = df_corr.corr()
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)
cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)
corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)


Inference :

1. Comparing Left, Right and Center with PE or not -- From the above correlation matrix it is evident that the column PE or Not is correlated with the following attributes in descending order - Left-sided > Right-sided > Center. 